<a href="https://colab.research.google.com/github/MagaliDrumare/Florence-2-Microsoft-/blob/main/Florence_2_Microsoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flash_attn timm einops

In [ ]:
pip install pillow

In [ ]:
import requests

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

prompt = "<OD>"

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt")

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    num_beams=3,
    do_sample=False
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)





In [ ]:
import torch
from PIL import Image, ImageDraw

parsed_answer = {'<OD>': {'bboxes': [[34.23999786376953, 160.0800018310547, 597.4400024414062, 371.7599792480469],
                                    [456.0, 97.68000030517578, 580.1599731445312, 261.8399963378906],
                                    [450.8800048828125, 276.7200012207031, 554.5599975585938, 370.79998779296875],
                                    [95.68000030517578, 280.55999755859375, 198.72000122070312, 371.2799987792969]],
                         'labels': ['car', 'door', 'wheel', 'wheel']}}

# Assuming 'image' is already loaded as a PIL Image object
draw = ImageDraw.Draw(image)

# Access bounding boxes and labels
bboxes = parsed_answer['<OD>']['bboxes']
labels = parsed_answer['<OD>']['labels']

for bbox, label in zip(bboxes, labels):
    top_left = (int(bbox[0] * image.width), int(bbox[1] * image.height))
    bottom_right = (int((bbox[0] + bbox[2]) * image.width), int((bbox[1] + bbox[3]) * image.height))

    print("Top-left:", top_left)
    print("Bottom-right:", bottom_right)

    draw.rectangle([top_left, bottom_right], outline="red", width=6)
    draw.text(top_left, label, fill="red")

display(image)  # Display the image with drawn boxes

In [ ]:
from PIL import Image, ImageDraw

# Your complete parsed_answer dictionary (please provide the full content)
parsed_answer = {'<OD>': {'bboxes': [[34.23999786376953, 160.0800018310547, 597.4400024414062, 371.7599792480469],
                                    [456.0, 97.68000030517578, 580.1599731445312, 261.8399963378906],
                                    [450.8800048828125, 276.7200012207031, 554.5599975585938, 370.79998779296875],
                                    [95.68000030517578, 280.55999755859375, 198.72000122070312, 371.2799987792969]], # Complete this line
                         'labels': ['car', 'door', 'wheel', 'wheel']}} # And this list

# Create a drawing object
draw = ImageDraw.Draw(image)

# Extract bounding boxes and labels
bboxes = parsed_answer['<OD>']['bboxes']
labels = parsed_answer['<OD>']['labels']

# Iterate and draw boxes
for bbox, label in zip(bboxes, labels):
    # Assuming normalized coordinates, convert to pixel coordinates
    top_left = (int(bbox[0] * image.width), int(bbox[1] * image.height))
    bottom_right = (int((bbox[0] + bbox[2]) * image.width), int((bbox[1] + bbox[3]) * image.height))

    # Draw the rectangle
    draw.rectangle([top_left, bottom_right], outline="red", width=2)

    # Add the label
    draw.text(top_left, label, fill="red")

# Display or save the image
display(image)  # If you are in a Jupyter Notebook
#image.save('image_with_boxes.jpg') # To save the image to a file